- By.ID	태그의 id값으로 추출
- By.NAME	태그의 name값으로 추출
- By.XPATH	태그의 경로로 추출
- By.LINK_TEXT	링크 텍스트값으로 추출
- By.PARTIAL_LINK_TEXT	링크 텍스트의 자식 텍스트 값을 추출
- By.TAG_NAME	태그 이름으로 추출
- By.CLASS_NAME	태그의 클래스명으로 추출
- By.CSS_SELECTOR	css선택자로 추출

xpath 문법

표현	설명
nodename	노드명이 "nodename"인 노드 선택
/	루트 노드로부터 선택
//	현재 노드로부터 문서상의 모든 노드를 조회
.	현재 노드 선택
..	현재 노드의 부모노드 선택
@	현재 노드의 속성 선택
 

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import re
# 추가 패키지
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# click() 대신 사용할 패키지
from selenium.webdriver.common.keys import Keys
# .send_keys(Keys.ENTER)

In [2]:
#google driver 확인용

url = 'https://smartstore.naver.com/ltdcircle/products/5715694688'
driver = webdriver.Chrome('diver/chromedriver.exe')
driver.get(url)

C:\Users\PiGiraffe0\AppData\Local\Temp\ipykernel_9100\101449453.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('diver/chromedriver.exe')


In [3]:
# 리뷰 크롤링 함수 생성

def review_data(lst : list) :
    rev_cnt = 0
    for review in driver.find_elements(By.CLASS_NAME , 'YEtwtZFLDz > span') :
        if review.text not in ['재구매' , '한달사용기' , 'BEST' ,'정기구독'] :
            rev_cnt = rev_cnt + 1
            print(f'{rev_cnt} ' , end = '')
            lst.append(review.text)
#            print('-'*60)

In [4]:
def review_total() :
    text = driver.find_element(By.CLASS_NAME , '_3HJHJjSrNK').text
    return int(re.sub('[\W]' , '' , text))

In [5]:
# naver_smartstore


def smartstore_review_data() :
    page = driver.find_element(By.CLASS_NAME , 'cv6id6JEkg > div > div > a.fAUKm1ewwo._2Ar8-aEUTq')
    page_cnt = 0
    while page.text != '' :
        page_cnt = page_cnt + 1
        wait_time = 2
        WebDriverWait(driver, wait_time).until(
                    EC.visibility_of_all_elements_located(
                        (By.CLASS_NAME,'_2389dRohZq')))
        
        print(f'page = {page_cnt}')
        review_data(result)
        print()
        time.sleep(1)
        if driver.find_element(By.XPATH , '//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[6]').text == 500 :
            break 
        else :
            page.click()
            time.sleep(0.5)
        
    print('end page')

In [6]:
# naver_brand


def naver_brand_review_data() :
    for page in driver.find_elements(By.CLASS_NAME , 'cv6id6JEkg > div > div > a') :
        wait_time = 2
        WebDriverWait(driver, wait_time).until(
                    EC.visibility_of_all_elements_located(
                        (By.CLASS_NAME,'_2389dRohZq')))
        try :
            if page.text not in ['이전' , '다음' , ''] :
                page.click()
                time.sleep(0.5)
                print()
                print(f'page = {page.text}')
                review_data(result)
                # time.sleep(1)
            elif page.text == '다음' :
                page.click()
                # print('stop')
                time.sleep(0.5)
        except Exception as e :
            print('end page')



In [7]:
def naver_brand_shop_crawling(object_name : str) :
    globals()['result'] = []
    if 'brand' in url :
        rev_tl = review_total() // 200
        if review_total() > 10000 :
            rev_tl = 50
        elif review_total() % 200 != 0 :
            rev_tl = rev_tl + 1
        for paging in range(rev_tl) :
            naver_brand_review_data()
    
    elif 'smartstore' in url :
        smartstore_review_data()
        
    elif 'shopping' in url :
        smartstore_review_data()
            
            
    print()
    print(f'크롤링한 리뷰 개수 : {len(result)}')
    
    sample_review = pd.DataFrame({'review' : result})
    sample_review.to_excel(f'crawling_test/{object_name}_naverbrand_crawling.xlsx' , index = False)
    
    print(f'crawling_test/{object_name}_naverbrand_crawling.xlsx 엑셀 저장')
    
    

In [174]:
# 작동 확인
naver_brand_shop_crawling('aaa')

page = 1
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 2
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 3
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 4
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 5
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 6
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 7
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 8
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 9
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 10
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 11
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 12
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 13
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 14
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 15
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 16
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 17
1 2 3 4

# 다량 크롤링 함수
### 아래의 urls에 상품 영문 명, 네이버 스마트스토어/브랜드숍 주소를 
### 순차적으로 기입할 것


In [20]:
urls = [
#     # 코카콜라 제로       34359
#     ['cocacola_zero' , 'https://brand.naver.com/cocacola/products/4660954096']
#     # 스프라이트 제로     5364
#     , ['sprite_zero' , 'https://brand.naver.com/cocacola/products/5432762713'] 
#     # 닥터페퍼 제로       6575
#     , ['drpeper_zero' , 'https://brand.naver.com/cocacola/products/7269583998']  
#     # 1am 스파클링        10746
#     , ['1am_sparkling' , 'https://smartstore.naver.com/pickydiet/products/5135269156']
#     # 부르르 제로 사이다  4585
#     , ['bururu_zero_cider' , 'https://brand.naver.com/ilhwa/products/5223711011']
#     # 부르르 제로 콜라    1905
#     , ['bururu_zero_cola' , 'https://brand.naver.com/ilhwa/products/5223714430']
#     # 웰치스 제로         5996
#     , ['welchs_zero' , 'https://brand.naver.com/nongshim/products/6506005017']
#     # 펩시콜라 제로       21795
#     , ['pepsi_zero' , 'https://brand.naver.com/lottechilsung/products/5770857283']
#     # 칠성사이다 제로     11278
#     , ['chilsung_cider' , 'https://brand.naver.com/lottechilsung/products/5334363460']
#     # 나랑드 사이다        2295
#     , ['narangd_cider' , 'https://smartstore.naver.com/donga-otsuka/products/5737533784']
#     # 씨그램               24255
#     , ['seagram' , 'https://brand.naver.com/cocacola/products/5443719795']
    # 트레비               62550
#     ['trevi' , 'https://smartstore.naver.com/ncutong/products/389678131']
    # 빅토리아 탄산수      98114
#     ['victoria' , 'https://brand.naver.com/woongjinfood/products/4839117051']
    # 이마트 노브랜드 스파클링 라임 1440
#   ['sparklinglime', 'https://shopping.naver.com/window-products/emart/6022030859?NaPm=ct%3Dlcdq1mdc%7Cci%3D669742d75db781386a4be6ef820f791f3d881870%7Ctr%3Dslsl%7Csn%3D4531621%7Chk%3Ddf601521295d033adfddde59d098d20b250cf4d0']
    #이마트 노브랜드 스파클링 백포도 1369
# ['sparkling_grape', 'https://shopping.naver.com/window-products/emart/6022031037?NaPm=ct%3Dlcdq2aag%7Cci%3Dcde5f336a8522fa9b774a922fae685ec76562362%7Ctr%3Dslsl%7Csn%3D4531621%7Chk%3Dd26652e3e2cb1a8a4493ed508069a34ec6a8448d']
    ['1am_sparkling','https://smartstore.naver.com/pickydiet/products/5135269051?NaPm=ct%3Dlcjdpyvk%7Cci%3D0zi000323%5FrxRMt8m1i8%7Ctr%3Dpla%7Chk%3D2befd9fcbbd8a3a08d612744a205b1c8cd3562c1']

    # 815콜라 제로 없음
    
]

In [21]:
def init_driver() :
    driver = webdriver.Chrome('diver/chromedriver.exe')
    return driver
def NBS_crawling(urls_code , driver) :
    for cnt in urls_code :
        print(f'{cnt[0]} 크롤링')
        globals()['url'] = cnt[1]
        print(url)
        driver.get(url)
        time.sleep(2)
        naver_brand_shop_crawling(cnt[0])
        print()
        print('-'*60)
        print()


In [23]:
# 코카제로 ~ 씨그램
# 정기구독 태그 확인
driver = init_driver()
NBS_crawling(urls , driver)

C:\Users\ohans\AppData\Local\Temp\ipykernel_30888\2454787338.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('diver/chromedriver.exe')


sparkling_grape 크롤링
https://shopping.naver.com/window-products/emart/6022031037?NaPm=ct%3Dlcdq2aag%7Cci%3Dcde5f336a8522fa9b774a922fae685ec76562362%7Ctr%3Dslsl%7Csn%3D4531621%7Chk%3Dd26652e3e2cb1a8a4493ed508069a34ec6a8448d
page = 1
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 2
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 3
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 4
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 5
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 6
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 7
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 8
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 9
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 10
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 11
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 12
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 13
1 2 3 4 5 6 7 8 9 10 11 12 13 14 

In [182]:
# 트레비
driver = init_driver()
NBS_crawling(urls , driver)

C:\Users\PiGiraffe0\AppData\Local\Temp\ipykernel_4952\2454787338.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('diver/chromedriver.exe')


trevi 크롤링
https://smartstore.naver.com/ncutong/products/389678131
page = 1
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 2
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 3
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 4
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 5
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 6
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 7
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 8
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 9
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 10
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 11
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 12
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 13
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 14
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 15
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 16
1 2

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 262
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 263
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 264
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 265
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 266
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 267
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 268
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 269
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 270
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 271
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 272
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 273
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 274
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 275
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 276
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
pag

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 522
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 523
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 524
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 525
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 526
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 527
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 528
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 529
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 530
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 531
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 532
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 533
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 534
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 535
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 536
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
pag

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x005AF243]
	(No symbol) [0x00537FD1]
	(No symbol) [0x0042D04D]
	(No symbol) [0x0042FD34]
	(No symbol) [0x0042FBE5]
	(No symbol) [0x0042FE80]
	(No symbol) [0x00457D27]
	(No symbol) [0x0047858C]
	(No symbol) [0x00452BFF]
	(No symbol) [0x00478804]
	(No symbol) [0x0048C9EB]
	(No symbol) [0x00478386]
	(No symbol) [0x0045163C]
	(No symbol) [0x0045269D]
	GetHandleVerifier [0x00849A22+2655074]
	GetHandleVerifier [0x0083CA24+2601828]
	GetHandleVerifier [0x00658C0A+619850]
	GetHandleVerifier [0x00657830+614768]
	(No symbol) [0x005405FC]
	(No symbol) [0x00545968]
	(No symbol) [0x00545A55]
	(No symbol) [0x0055051B]
	BaseThreadInitThunk [0x774C00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77877BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77877B8E+238]


In [186]:
# trevi 오류 밑 작업 + smartstore 함수 개선

sample_frm = pd.DataFrame({'review' : result})

In [219]:
sf = sample_frm[11729 : ].reset_index(drop = True)
sf[:10000].to_excel(f'crawling_test/trevi_naverbrand_crawling.xlsx' , index = False)

In [236]:
# 빅토리아
# 빅토리아 이후 정기구독 태그 필터링 
driver = init_driver()
NBS_crawling(urls , driver)

C:\Users\PiGiraffe0\AppData\Local\Temp\ipykernel_4952\2454787338.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('diver/chromedriver.exe')


victoria 크롤링
https://brand.naver.com/woongjinfood/products/4839117051

page = 1
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 2
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 3
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 4
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 5
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 6
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 7
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 8
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 9
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 10
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 11
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 12
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 13
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 14
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 15
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 1

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 262
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 263
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 264
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 265
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 266
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 267
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 268
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 269
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 270
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 271
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 272
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 273
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 274
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 275
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
page = 276
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
pag